In [1]:
%cd ../..

/workspace/transactions/tabsyn-concat


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import json
from pathlib import Path
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np

from utils_train import preprocess, TabularDataset
from tabsyn.vae.model import Model_VAE, Encoder_model, Decoder_model

In [3]:
LR = 1e-3
WD = 0
D_TOKEN = 4
TOKEN_BIAS = True

N_HEAD = 1
FACTOR = 32
NUM_LAYERS = 2

In [4]:
dataname = "datafusion_with_id"
data_dir = f'data/{dataname}'
ckpt_dir = f"tabsyn/vae/ckpt/{dataname}"

info_path = f'data/{dataname}/info.json'
with open(info_path, 'r') as f:
    info = json.load(f)
    
device = "cuda:3"

In [5]:
info

{'name': 'datafusion',
 'task_type': 'binclass',
 'header': 'infer',
 'column_names': ['user_id',
  'days_since_first_tx',
  'mcc_code',
  'currency_rk',
  'transaction_amt',
  'customer_age',
  'time_diff_days',
  'dummy_binclass'],
 'num_col_idx': [1, 4, 6],
 'cat_col_idx': [0, 2, 3, 5],
 'target_col_idx': [7],
 'file_type': 'csv',
 'data_path': 'data/datafusion_with_id/preprocessed_with_id_train.csv',
 'test_path': None,
 'column_info': {'1': {},
  'type': 'categorical',
  'max': 90.94725694444443,
  'min': 0.0,
  '4': {},
  '6': {},
  '0': {},
  'categorizes': [0, 1],
  '2': {},
  '3': {},
  '5': {},
  '7': {}},
 'train_num': 9356053,
 'test_num': 1039562,
 'idx_mapping': {'0': 3,
  '1': 0,
  '2': 4,
  '3': 5,
  '4': 1,
  '5': 6,
  '6': 2,
  '7': 7},
 'inverse_idx_mapping': {'3': 0,
  '0': 1,
  '4': 2,
  '5': 3,
  '1': 4,
  '6': 5,
  '2': 6,
  '7': 7},
 'idx_name_mapping': {'0': 'user_id',
  '1': 'days_since_first_tx',
  '2': 'mcc_code',
  '3': 'currency_rk',
  '4': 'transaction_am

# Get latent codes

In [6]:
X_num, X_cat, categories, d_numerical = preprocess(data_dir, task_type = info['task_type'])

No NaNs in numerical features, skipping


In [1]:
categories

NameError: name 'categories' is not defined

In [8]:
d_numerical = 2
categories.pop(1)
categories[1] = 332
d_numerical, categories

(2, [2, 332, 4, 4])

In [9]:
# X_num[0][:10]

In [10]:
print(X_cat[0].shape)
X_cat[0][:40, 4]

(9356053, 5)


array([0, 1, 0, 0, 1, 0, 1, 2, 2, 2, 3, 3, 0, 0, 3, 3, 3, 1, 0, 1, 2, 1,
       0, 1, 0, 3, 1, 3, 3, 0, 0, 3, 2, 1, 0, 2, 0, 1, 0, 3])

In [11]:
age_balance = dict()
for c in [0, 1, 2, 3]:
    age_balance[c] = np.sum(X_cat[0][:, 4] == c)
    print(f'{c} : {age_balance[c]} : {age_balance[c] / len(X_cat[0])}')

0 : 2440056 : 0.26079971971086524
1 : 2103250 : 0.22480099246979468
2 : 2229064 : 0.2382483297176705
3 : 2583683 : 0.27615095810166956


In [13]:
X_train_num, _ = X_num
X_train_cat, _ = X_cat

X_train_num, X_test_num = X_num
X_train_cat, X_test_cat = X_cat

X_train_num, X_test_num = torch.tensor(X_train_num).float(), torch.tensor(X_test_num).float()
X_train_cat, X_test_cat =  torch.tensor(X_train_cat), torch.tensor(X_test_cat)

train_data = TabularDataset(X_train_num.float(), X_train_cat)
test_data = TabularDataset(X_test_num.float(), X_test_cat)

batch_size = 2**15
train_loader = DataLoader(
    train_data,
    batch_size=batch_size,
    num_workers=4,
    drop_last=False,
)
test_loader = DataLoader(
    test_data,
    batch_size=batch_size,
    num_workers=4,
    drop_last=False,
)

In [14]:
model = Model_VAE(NUM_LAYERS, d_numerical, categories, D_TOKEN, n_head = N_HEAD, factor = FACTOR, bias = True)
model = model.to(device)

pre_encoder = Encoder_model(NUM_LAYERS, d_numerical, categories, D_TOKEN, n_head = N_HEAD, factor = FACTOR).to(device)
pre_decoder = Decoder_model(NUM_LAYERS, d_numerical, categories, D_TOKEN, n_head = N_HEAD, factor = FACTOR).to(device)

pre_encoder.eval()
pre_decoder.eval()

self.category_embeddings.weight.shape=torch.Size([342, 4])
self.category_embeddings.weight.shape=torch.Size([342, 4])


Decoder_model(
  (VAE_Decoder): Transformer(
    (layers): ModuleList(
      (0): ModuleDict(
        (attention): MultiheadAttention(
          (W_q): Linear(in_features=4, out_features=4, bias=True)
          (W_k): Linear(in_features=4, out_features=4, bias=True)
          (W_v): Linear(in_features=4, out_features=4, bias=True)
        )
        (linear0): Linear(in_features=4, out_features=128, bias=True)
        (linear1): Linear(in_features=128, out_features=4, bias=True)
        (norm1): LayerNorm((4,), eps=1e-05, elementwise_affine=True)
      )
      (1): ModuleDict(
        (attention): MultiheadAttention(
          (W_q): Linear(in_features=4, out_features=4, bias=True)
          (W_k): Linear(in_features=4, out_features=4, bias=True)
          (W_v): Linear(in_features=4, out_features=4, bias=True)
        )
        (linear0): Linear(in_features=4, out_features=128, bias=True)
        (linear1): Linear(in_features=128, out_features=4, bias=True)
        (norm1): LayerNorm((

In [15]:
ckpt = torch.load(f"{ckpt_dir}/model.pt")
model.load_state_dict(ckpt)

<All keys matched successfully>

In [16]:
model.eval()

Model_VAE(
  (VAE): VAE(
    (Tokenizer): Tokenizer(
      (category_embeddings): Embedding(342, 4)
    )
    (encoder_mu): Transformer(
      (layers): ModuleList(
        (0): ModuleDict(
          (attention): MultiheadAttention(
            (W_q): Linear(in_features=4, out_features=4, bias=True)
            (W_k): Linear(in_features=4, out_features=4, bias=True)
            (W_v): Linear(in_features=4, out_features=4, bias=True)
          )
          (linear0): Linear(in_features=4, out_features=128, bias=True)
          (linear1): Linear(in_features=128, out_features=4, bias=True)
          (norm1): LayerNorm((4,), eps=1e-05, elementwise_affine=True)
        )
        (1): ModuleDict(
          (attention): MultiheadAttention(
            (W_q): Linear(in_features=4, out_features=4, bias=True)
            (W_k): Linear(in_features=4, out_features=4, bias=True)
            (W_v): Linear(in_features=4, out_features=4, bias=True)
          )
          (linear0): Linear(in_features=4,

In [17]:
pre_encoder.load_weights(model)
pre_decoder.load_weights(model)

In [18]:
pre_encoder.eval()
pre_decoder.to(device)

Decoder_model(
  (VAE_Decoder): Transformer(
    (layers): ModuleList(
      (0): ModuleDict(
        (attention): MultiheadAttention(
          (W_q): Linear(in_features=4, out_features=4, bias=True)
          (W_k): Linear(in_features=4, out_features=4, bias=True)
          (W_v): Linear(in_features=4, out_features=4, bias=True)
        )
        (linear0): Linear(in_features=4, out_features=128, bias=True)
        (linear1): Linear(in_features=128, out_features=4, bias=True)
        (norm1): LayerNorm((4,), eps=1e-05, elementwise_affine=True)
      )
      (1): ModuleDict(
        (attention): MultiheadAttention(
          (W_q): Linear(in_features=4, out_features=4, bias=True)
          (W_k): Linear(in_features=4, out_features=4, bias=True)
          (W_v): Linear(in_features=4, out_features=4, bias=True)
        )
        (linear0): Linear(in_features=4, out_features=128, bias=True)
        (linear1): Linear(in_features=128, out_features=4, bias=True)
        (norm1): LayerNorm((

In [19]:
latent_codes = []
client_ids = []
abs_times = []
age_categories = []

with torch.inference_mode():
    for batch_num, batch_cat in tqdm(train_loader):
        batch_num = batch_num.to(device)
        batch_cat = batch_cat.to(device)
        abs_time, batch_num = batch_num[:, 0], batch_num[:, 1:]
        client_id, batch_cat, age_category = batch_cat[:, 1], batch_cat[:, [0, 2, 3, 4]], batch_cat[:, [4]]
        
        train_z = pre_encoder(batch_num, batch_cat).cpu().numpy()
        latent_codes.append(train_z)
        client_ids.append(client_id.cpu().numpy())
        abs_times.append(abs_time.cpu().numpy())
        age_categories.append(age_category.cpu().numpy())

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 286/286 [00:23<00:00, 11.92it/s]


In [20]:
train_z = np.concatenate(latent_codes)
train_client_ids = np.concatenate(client_ids)
train_abs_times = np.concatenate(abs_times)
train_age_categories = np.concatenate(age_categories)

In [21]:
train_z.shape

(9356053, 7, 4)

In [22]:
train_age_categories[:20]

array([[0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [2],
       [2],
       [2],
       [3],
       [3],
       [0],
       [0],
       [3],
       [3],
       [3],
       [1],
       [0],
       [1]])

In [23]:
latent_codes = []
client_ids = []
abs_times = []
age_categories = []

with torch.inference_mode():
    for batch_num, batch_cat in tqdm(test_loader):
        batch_num = batch_num.to(device)
        batch_cat = batch_cat.to(device) # batch_cat[:, 4] - age, понять, чему соответствует в изначальных фичах. 
        # groupby + количество штук - сравнить с изначальными датафреймом (train)
        abs_time, batch_num = batch_num[:, 0], batch_num[:, 1:]
        client_id, batch_cat, age_category = batch_cat[:, 1], batch_cat[:, [0, 2, 3, 4]], batch_cat[:, [4]]
        
        z = pre_encoder(batch_num, batch_cat).cpu().numpy()
        latent_codes.append(z)
        client_ids.append(client_id.cpu().numpy())
        abs_times.append(abs_time.cpu().numpy())
        age_categories.append(age_category.cpu().numpy())

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:03<00:00,  9.18it/s]


In [24]:
test_z = np.concatenate(latent_codes)
test_client_ids = np.concatenate(client_ids)
test_abs_times = np.concatenate(abs_times)
test_age_categories = np.concatenate(age_categories)

In [25]:
test_z.shape

(1039562, 7, 4)

In [26]:
test_age_categories.shape

(1039562, 1)

In [27]:
np.save(f'{ckpt_dir}/train_z.npy', train_z)
np.save(f'{ckpt_dir}/train_client_ids.npy', train_client_ids)
np.save(f'{ckpt_dir}/train_abs_times.npy', train_abs_times)
np.save(f'{ckpt_dir}/train_age_categories.npy', train_age_categories)

In [28]:
np.save(f'{ckpt_dir}/test_z.npy', test_z)
np.save(f'{ckpt_dir}/test_client_ids.npy', test_client_ids)
np.save(f'{ckpt_dir}/test_abs_times.npy', test_abs_times)
np.save(f'{ckpt_dir}/test_age_categories.npy', test_age_categories)

# Sort codes

In [63]:
import pandas as pd
from IPython.display import display

In [6]:
train_z = np.load(f'{ckpt_dir}/train_z.npy')
train_client_ids = np.load(f'{ckpt_dir}/train_client_ids.npy')
train_abs_times = np.load(f'{ckpt_dir}/train_abs_times.npy')

In [7]:
df_seqs = pd.DataFrame(dict(idx=train_client_ids, order=train_abs_times))
df_seqs

,idx,order
0,14315,-0.489501
1,17935,-0.566783
2,26622,-1.252068
3,48255,0.175591
4,36048,-0.455435
...,...,...
9356048,41529,0.143390
9356049,5793,-0.500177
9356050,39608,0.657490
9356051,5910,0.346575


In [8]:
df_seqs = df_seqs.sort_values(["idx", "order"])
df_seqs

,idx,order
7764011,0,-3.014032
3805364,0,-1.187927
5330317,0,-0.701916
587580,0,-0.652003
7456754,0,-0.186872
...,...,...
4961325,76799,1.861384
3090022,76799,1.862275
1721820,76799,1.878158
7467402,76799,1.878258


In [9]:
df_seqs.groupby("idx").size().describe()

count    76800.000000
mean       121.823607
std        133.992507
min          5.000000
25%         30.000000
50%         73.000000
75%        167.000000
max       1357.000000
dtype: float64

In [58]:
dev = df_seqs[df_seqs.idx <= 1]
dev

,idx,order
7764011,0,-3.014032
3805364,0,-1.187927
5330317,0,-0.701916
587580,0,-0.652003
7456754,0,-0.186872
...,...,...
6925063,1,1.374727
8270442,1,1.374869
220895,1,1.377111
5817231,1,2.651878


In [75]:
length = 12

In [86]:
def get_slices(df):
    w = filter(lambda df: len(df) == length, df.rolling(window=length))
    indices = list(map(lambda df: df.index.values, w))
    if not indices:
        return pd.DataFrame(columns=list(range(length)), dtype=int)
    return pd.DataFrame(np.stack(indices))

In [87]:
idx = dev.groupby("idx", group_keys=False).apply(get_slices, include_groups=False).values

In [88]:
train_z[idx].shape

(72, 12, 7, 4)

In [70]:
train_z.shape

(9356053, 7, 4)